***Public AI***
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 9. CNN Basis**
# Section 1. 합성곱 연산이란

### _Objective_
1. 기존의 DNN 방식에서 FullyConnectedLayer(이하 FC)  을 통해 이미지 처리를 하는 법을 배워 봅니다

2. 이미지 처리에 딥러닝을 적용시에 어떤 문제점이 발생되는지 파악해 봅니다. 

2. 이미지의 기본적인 특성을 배워 보고 이미지의 특성을 활용해 딥러닝을 적용하는 법에 대해 배워 봅니다. 

3. 대부분의 합성곱 연산은 4차원의 데이터를 다룹니다. 이를 어떻게 연산하는 지를 배워봅니다. <br>

4. 합성곱 연산의 연산량을 줄여주는 풀링 연산에 대해 배워봅니다.<br> 
  

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2

<br><br>

# \[ 1. FC Layer 을 활용해 이미지 처리 하기 \]

----

----

> 이미지를 한줄로 Flatten 한 후 기존 방법 처럼 FC Layer 을 연결합니다

In [ ]:
# import needed library 
import tensorflow as tf 
import tensorflow.keras as keras 
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import Model 
from tensorflow.keras.utils import to_categorical
tf.get_default_graph()

# fashion mnist 가져오기
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images/ 255.
test_images = test_images/ 255.

# model 
inputs = Input(shape=(28,28))

# ** 이렇게 이미지를 한줄로 편 후 Deep Learning Model 에 연결합니다.** 
flat_inputs = Flatten()(inputs)

dense1 = Dense(128, activation='relu')(flat_inputs)
dense2 = Dense(128, activation='relu')(dense1)
pred = Dense(10, activation='softmax')(dense2)

model = Model(inputs, pred)
# optimizer 

train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)
model.compile('sgd', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
hist = model.fit(train_images, train_labels,
                 batch_size=64, validation_split=0.1, epochs=10)

### Image을 FC Layer 에 연결시 단점 
![Imgur](https://i.imgur.com/r5SO4i2.png)

- 이미지를 구별하기 위해 필요하지 않는 정보들을 weights 에 연결해야 해 weights 갯수가 많이 필요하다는 단점이 있습니다. 


# \[ 2. 동물이 사물의 형태를 인지하는 방법 \]

- 동물들은 간단한 저차원 (점, 선) 특징들을 추출한 후 추출된 특징들을 조합해 점차 복잡한 이미지를 구성합니다. 

[Receptive fields of single neurones in the cat's striate cortex
D. H. Hubel and T. N. Wiesel](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1363130/)

![Imgur](https://i.imgur.com/rf3kDoy.png)

# \[ 3. 이미지의 특징 \]
![Imgur](https://i.imgur.com/FgoyoPW.png)
- 이미지는 픽셀과 인접해 있는 픽셀간의 값의 차이를 통해 이미지의 특징을 표현할수 있습니다.
- 국소적인 특징들을 조합해 복잡한 특징들을 나타낼수 있습니다.

# \[ 4. 이미지의 구조 \]

> 이미지가 어떻게 컴퓨터에서 보여지고 구조화 되어 있는지 확인해 보겠습니다.

**컴퓨터 비전**은 컴퓨터를 이용하여 정지영상 또는 동영상으로부터 의미있는 정보를 추출하는 방법을 연구하는 학문입니다. 이미지를 분석하기 위해선 그에 대한 정보가 필요하기 때문입니다. 컴퓨터 비전을 간단하게 정리하면 사람이 눈으로 보고 인지하는 작업을 컴퓨터가 동등하게 수행할 수 있게끔 연구하는 학문이라고 할 수 있습니다. 이런 작업이 우리에게 왜 필요할까요?

컴퓨터 비전은 이미지로부터 의미있는 정보를 추출하는 것을 다룹니다. 사람이 사물을 보고 무엇인지 인지하고 그에 대한 정보를 추출하는 것은 매우 쉬운 작업이지만, 컴퓨터가 사물을 인식할 수 있도록 하는 일은 매우 어렵습니다. 이를 예시를 통해 살펴보겠습니다.

![](https://i.imgur.com/0UOiXGp.png)

우리 인간은 눈으로 사과와 토마토를 각각 인식할 수 있습니다. 그러나 컴퓨터의 경우는 인간의 경우와 전혀 다릅니다. <br>
첫번째 그림과 같은 사과 사진을 컴퓨터의 input으로 주고, 이 사진의 객체를 사과라고 인식하는 문제에 대해 생각해 보겠습니다. 흰 배경을 제외한 가운데 영역에 빨간색 성분이 많이 있고 둥근 윤곽을 가지면 사과라고 인식하게끔 프로그램을 만들면 빨간 사과를 인식할 수 있을 것입니다. 그러나 두번째 그림처럼 초록색 사과도 함께 인식해야 한다면 사과의 색상정보에 초록색도 추가해야 합니다. 만약 세번째 그림처럼 빨간색 토마토가 입력으로 들어오면 상황은 더 복잡해집니다. 색상정보와 윤곽선 정보만으로는 사과와 토마토가 구분이 되지 않으므로 꼭지의 모양까지 고려해야 합니다. 네번째 그림처럼 배경이 단순하지 않고 여러 과일이 겹쳐 있는 경우에는 인식이 더더욱 어려워지겠죠. 

이러한 문제들을 컴퓨터 비전에서는 영상으로부터 유용한 정보를 추출하고, 이를 조합하여 결과를 유추합니다. 컴퓨터 비전에서 주로 사용하는 영상정보는 밝기, 색상, 모양 등이 있으며, 이러한 정보와 머신러닝, 딥러닝 알고리즘을 함께 사용하여 사물을 인지할 수 있습니다. 그러나 영상으로부터 이러한 정보들을 추출하는 것은 쉬운일이 아닙니다. 예를들어 배경과 객체를 어떻게 구분해야 하는지, 빨간색을 판단하기 위해서는 어떤 수식을 사용해야 하는지, 둥근 윤곽을 판단하기 위해서는 어떤 알고리즘을 사용해야 하는지 등을 결정하는 것은 쉬운 문제가 아닙니다. 앞으로 수업시간에는 영상으로부터 유용한 정보를 추출하는 방법과 추출된 정보를 효과적으로 사용하는 방법들에 대해 다루게 될것입니다.

## What is Image?

아래 이미지는 조르주 쇠라의 '그랑트 자트 섬의 센 강, 봄' 입니다. 인상주의 이후 선보인 '점묘법'에 의해 그려진 그림으로, 자잘한 점을 캔버스 표면에 나란히 찍어가며 풍경을 묘사하고 있습니다. 

![](https://gscaltexmediahub.com/wp-content/uploads/2018/10/GSC_BS_MH_gs-calender-2019-04_20181026_01.jpg)

컴퓨터가 이미지를 이해하는 **픽셀**의 아이디어가 위와 같은 점묘법에서 출발했습니다. 여기서 픽셀은 이미지를 이루는 낱낱의 점들을 말하고, 하나의 픽셀은 하나의 밝기 또는 색상을 표현하며, 이러한 픽셀이 모여서 2차원 영상을 구성합니다. 컴퓨터는 이미지를 가장 먼저 픽셀로 이해하며, 이 픽셀은 일련의 숫자로 그 값을 표시합니다. 예를들어 컴퓨터의 입장에서 1024 * 768의 해상도를 가진 이미지는 1024 * 768 = 786432개의 점, 즉 786432개의 픽셀을 포함하는 1024개의 열과 768 개의 행이 있는 **행렬형태**로 존재하게 됩니다.

![](https://i.imgur.com/goyV2hO.png)

컴퓨터 비전에서는 주로 grayscale image와 truecolor image를 사용합니다. Grayscale image는 흑백 영상과 같이 밝기 정보로만 구성된 영상을 의미하고, 회색조 영상이라고도 합니다. 반면에 다양한 색상을 표현할 수 있는 영상을 컬러 영상이라고합니다. 그레이스케일 영상은 밝기 정보를 256단계로 구분하여 표현합니다. 그레이스케일 영상에서 하나의 픽셀은 0부터 255사이의 정수 값으로 256단계를 표현합니다. 가장 밝음을 나타내는 255에서, 빛이 없음을 나타내는 0까지의 그라데이션으로 밝기를 표현합니다. 컬러영상은 기본적으로 R, G, B 세 개의 색상 성분 조합으로 픽셀 값을 표현합니다. 한 픽셀당 R채널, G채널, B채널의 각각의 배열에, 각각의 색상 성분은 0부터 255사이의 정수 값으로 표현되며 이들의 조합으로 이미지의 색을 나타냅니다.

![](https://i.imgur.com/XiWSz2n.png)

---
⊙ Copyright(c) 2020 by PublicAI. All rights reserved <br>
All pictures, codes, writings cannot be copied without permission. <br>
Writen by PAI(info@publicai.co.kr) <br>
last updated on 2020/01/4 <br>

---